In [1]:
# coding: utf-8
from typing import Dict, Optional

import pandas as pd
import plotly.express as px

from data.data_loader import PykrxDataLoader
from simulation.account import Account
from simulation.broker import Broker
from simulation.metric import cagr, mdd, sharpe_ratio, sortino_ratio
from simulation.utility import get_lookback_fromdate, rebalance

## 1. 팩터 투자

### 1.1. 유틸리티 함수 정의

In [2]:
def date_adjust(index_df: pd.DataFrame, df: pd.DataFrame):
    new_index = downsample_df(index_df).index
    # 원래 인덱스에 새 인덱스 삽입
    df.index = df.index.map(lambda x: x.replace(day=new_index[(new_index.year == x.year) & (new_index.month == x.month)][0].day))
    return df

def downsample_df(df: pd.DataFrame) -> pd.DataFrame:
    data = df.copy()
    data.index = pd.to_datetime(data.index)
    data['date'] = data.index
    data = data.resample('M').apply('last')

    return data.set_index(pd.DatetimeIndex(data.date)).drop(columns=['date'])

### 1.2. 팩터 전략 정의하기

#### 1.2.1. 모멘텀(상대) 전략 정의하기

In [3]:
def calculate_momentum(ohlcv_data: pd.DataFrame,
                       lookback_period: int,
                       skip_period: int) -> pd.DataFrame:
    # 데이터 재구조화
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(['ticker', 'date']).unstack(level=0)['close']

    # 팩터 계산(모멘텀)
    momentum_data = data.shift(periods=skip_period).rolling(window=lookback_period).apply(lambda x: x[-1] / x[0] - 1)

    return momentum_data

#### 1.2.2. 가치(PER, PBR) 및 배당 전략 정의하기

### 1.3. 시뮬레이션을 위한 함수 정의하기

In [4]:
def get_factor_weight(factor_data: pd.DataFrame,
                      buying_ratio: float,
                      strategy_name: str) -> Optional[Dict]:
    # 데이터 중 결측치가 있는지 확인함
    if factor_data.isnull().values.any():
        return None

    # 매수 주식 선정
    reverse = {'per', 'pbr', 'small', 'lowvol'}
    ratio = buying_ratio if strategy_name in reverse else 1 - buying_ratio
    top_quantile = factor_data.quantile(ratio)
    if strategy_name in reverse:
        stocks_to_buy = factor_data[factor_data <= top_quantile].index.to_list()
    else:
        stocks_to_buy = factor_data[factor_data >= top_quantile].index.to_list()

    # 주식 비율 할당
    weights = 1 / len(stocks_to_buy) if stocks_to_buy else 0
    portfolio = {ticker: weights if ticker in stocks_to_buy else 0.0 for ticker in factor_data.index}

    return portfolio